In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments
from datasets import load_dataset

/home/hosman/anaconda3/envs/torchEnv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-02-15 17:54:57.311983: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-15 17:54:57.322513: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739631297.333230  351737 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739631297.336533  351737 cuda_blas.cc

In [5]:
# 1. Model ve Tokenizer'ı Yükle
checkpoint = "HuggingFaceTB/SmolLM2-360M-Instruct"  # Mistral 7B modeli


In [12]:


device = "cuda" # for GPU usage or "cpu" for CPU usage
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
# for multiple GPUs install accelerate and do `model = AutoModelForCausalLM.from_pretrained(checkpoint, device_map="auto")`
model = AutoModelForCausalLM.from_pretrained(checkpoint).to(device)

messages = [{"role": "system", "content": "Senin adın Mia"},{"role": "user", "content": "Adın ne?"}]
input_text=tokenizer.apply_chat_template(messages, tokenize=False)
print(input_text)
inputs = tokenizer.encode(input_text, return_tensors="pt").to(device)
outputs = model.generate(inputs, max_new_tokens=512, temperature=0.2, top_p=0.9, do_sample=True)

outputs

<|im_start|>system
Senin adın Mia<|im_end|>
<|im_start|>user
Adın ne?<|im_end|>



tensor([[    1,  9690,   198, 27025,   254,   493, 20335,    94, 21230,     2,
           198,     1,  4093,   198,  4503, 20335,    94,   420,    47,     2,
           198,     1,   520,  9531,   198,  4503, 20335,    94,   420,    47,
             2]], device='cuda:0')

In [13]:
print(tokenizer.decode(outputs[0]))

<|im_start|>system
Senin adın Mia<|im_end|>
<|im_start|>user
Adın ne?<|im_end|>
<|im_start|>assistant
Adın ne?<|im_end|>
